<a href="https://colab.research.google.com/github/dvoils/neural-network-experiments/blob/main/energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Hopfield proposed that certain computational properties useful to organisms and computers can emerge **collectively** from large assemblies of simple interacting units (neurons). Instead of requiring complex circuitry, emergent computation arises spontaneously, analogous to physical systems such as magnetic domains or fluid vortices. This paper presents a model that exhibits **content-addressable memory**, error correction, generalization, and categorization---all emerging from dynamics rather than programmed instruction.


## Content-Addressable Memory and Dynamics

Let a memory be represented as a point $\mathbf{X} \in \mathbb{R}^N$. In certain physical systems (e.g., Ising models), dynamics defined by gradient descent in energy space drive states toward stable attractors:

$$
\frac{d\mathbf{X}}{dt} = -\nabla E(\mathbf{X})
$$

This system acts as a **content-addressable memory** if every partial or noisy input state $\mathbf{X}' \approx \mathbf{X}_a$ flows toward a stable point $\mathbf{X}_a$. Hopfield demonstrates that such dynamics can recover full memories from fragments.

## The Hopfield Model

Each of the $N$ neurons is binary:

$V_i \in \{0, 1\} \quad \text{or equivalently} \quad s_i = 2V_i - 1 \in \{-1, +1\}$

Neurons update asynchronously using the rule:

$$
V_i \leftarrow
\begin{cases}
1 & \text{if } \sum_j T_{ij} V_j > U_i \\
0 & \text{otherwise}
\end{cases}
$$

where $T_{ij}$ is the synaptic strength, and $U_i$ is the threshold (often taken to be 0).


## Hebbian Learning Rule

To store a set of binary patterns $\{\mathbf{V}^s\}_{s=1}^n$, Hopfield applies the Hebbian learning rule:

$$
T_{ij} = \sum_{s=1}^{n} (2V_i^s - 1)(2V_j^s - 1), \quad T_{ii} = 0
$$

This rule causes each stored pattern $\mathbf{V}^s$ to become a local minimum (attractor) in energy space:

$$
E = -\frac{1}{2} \sum_{i \neq j} T_{ij} V_i V_j
$$

Updating any neuron causes the energy to decrease:

$$
\Delta E = -\Delta V_i \sum_j T_{ij} V_j
$$

Thus, asynchronous updates guarantee convergence to a stable state.

## Capacity and Error Correction

* For $N$ neurons, the network can stably store about $0.15N$ random patterns before retrieval degrades.
* Noise in the system is modeled as Gaussian, leading to error probability:

$$
P = \frac{1}{\sqrt{2\pi}} \int_x^\infty e^{-t^2/2} dt
$$

* Simulations confirm recall is accurate for low pattern count and degrades as $n$ approaches $0.15N$.

## Categorization and Familiarity

* **Generalization**: The system categorizes ambiguous inputs by converging to the closest memory.
* **Familiarity**: High activation rates during convergence can indicate whether a pattern is familiar.
* **Categorical recall**: Close patterns may collapse into a shared attractor (useful for pattern completion).

## Extensions and Properties

* **Clipped Weights**: Even if $T_{ij} \in \{-1, 0, +1\}$, performance only degrades slightly.
* **Asymmetry**: Even non-symmetric $T_{ij} \neq T_{ji}$ can yield metastable attractors.
* **Forgetting**: Saturating synaptic strength (e.g., $T_{ij} \in [-3, 3]$) introduces natural forgetting.
* **Sequence Recall**: Adding asymmetric terms can allow short sequences $V^1 \to V^2 \to V^3 \to \dots$.

## Biological Plausibility

* Real neurons exhibit firing rates that approximate binary thresholds.
* Hebbian learning ($\Delta T_{ij} \propto V_i V_j$) is biologically plausible.
* Delay and stochasticity are modeled via asynchronous updates.


## Conclusions

Hopfield demonstrates that associative memory and pattern completion can emerge as collective properties of simple neuron-like elements. These results suggest:

1. Neural computation does not require complex sequential logic.
2. Distributed systems can perform robust parallel computation.
3. The brain may exploit such physical dynamics for memory, recognition, and decision-making.
4. Hardware implementations (e.g., neuromorphic chips) could benefit from these ideas.

**Key Concepts**: attractor dynamics, energy minimization, Hebbian learning, content-addressable memory, neural computation, error correction, categorization.

**Citations**: J.J. Hopfield, PNAS, Vol. 79, pp. 2554–2558, 1982.


# 1. Models of Neural Networks

*A Statistical Mechanics Approach to Associative Memory*

## 1.1 Binary-State Neurons and Spin Variables

In this section, Amit, Gutfreund, and Sompolinsky present a powerful analogy between neural networks and disordered spin systems in statistical mechanics. This framework formalizes the behavior of networks such as the Hopfield model and the earlier Little model by treating each neuron as a binary unit. Neurons are modeled as Ising-like spins that can be in one of two discrete states: $S_i = +1$ indicates that a neuron is firing (active), and $S_i = -1$ indicates that it is not firing (inactive). These binary states correspond to the presence or absence of electrochemical activity over short time intervals, typically on the order of milliseconds.

The state of the entire network at a given time $t$, associated with a particular memory pattern labeled by $\alpha$, is represented by the vector:

$$
| \alpha, t \rangle = | S_1^\alpha, S_2^\alpha, \ldots, S_N^\alpha; t \rangle
\tag{1.1}
$$

This vector captures the configuration of all $N$ neurons, where each $S_i^\alpha$ is the state of the $i$-th neuron in memory pattern $\alpha$. The full network state exists in a $2^N$-dimensional binary phase space, with each configuration representing a potential microstate of the system. These memory states form the foundational units of computation and recall in the model.

## 1.2 Synaptic Connectivity and Postsynaptic Potentials

Neurons are coupled through synaptic junctions with connection strengths $J_{ij}$, which determine the influence of neuron $j$ on neuron $i$. These interactions may be excitatory (positive $J_{ij}$) or inhibitory (negative $J_{ij}$). During a short integration period, each neuron aggregates input from other neurons. The total **postsynaptic potential** $V_i$ received by neuron $i$ is defined as:

$$
V_i = \sum_j J_{ij} (S_j + 1)
\tag{1.2}
$$

This formulation maps the spin values $S_j \in \{-1, +1\}$ into the domain $\{0, 2\}$, ensuring that only active neurons contribute to the synaptic sum. Specifically, an inactive presynaptic neuron ($S_j = -1$) contributes nothing to $V_i$, while an active one ($S_j = +1$) contributes $2J_{ij}$. This encoding preserves biological plausibility, as silent neurons do not influence postsynaptic potentials.

## 1.3 Thresholding and Neuronal Stability

Once the total potential $V_i$ is computed, it is compared to a threshold value $U_i$. The difference $h_i = V_i - U_i$ is known as the **molecular field** or local input field. A neuron is said to be stable when its state $S_i$ is aligned with its molecular field. This condition is expressed as:

$$
S_i h_i = S_i (V_i - U_i) > 0
\tag{1.3}
$$

If this inequality is satisfied, the neuron is energetically stable and will not change state. If the inequality is violated, the neuron is misaligned with its input field and will flip its state in the next update. This condition ensures that the network only evolves through **state transitions that reduce the system's energy**, pushing the system toward more stable configurations.

## 1.4 The Hamiltonian and Energy Descent

The global behavior of the network is governed by a scalar energy function, known as the **Hamiltonian**. It measures the total energy of the system in a given configuration. The Hamiltonian is defined as:

$$
H = -\sum_i h_i S_i = -\frac{1}{2} \sum_{i,j} J_{ij} S_i S_j
\tag{1.4}
$$

The first form of the Hamiltonian expresses the alignment between each neuron and its local input field. The second form arises under the assumption that the synaptic weights are symmetric, $J_{ij} = J_{ji}$, and avoids double-counting pairwise interactions with a factor of $\frac{1}{2}$. When threshold potentials are chosen such that $U_i = \sum_j J_{ij}$, any constant external field contributions are eliminated, and the energy is entirely determined by inter-neuronal interactions.

This energy function serves as a Lyapunov function for the system: the dynamics defined by the update rule in Equation (1.3) guarantee that each neuron flip reduces or leaves unchanged the value of $H$. Therefore, the system follows a path of **monotonic energy descent** and converges to a local minimum of the Hamiltonian. These minima correspond to **stable memory configurations**.

## 1.5 Memory Storage via Hebbian Learning

The storage of memory patterns in the network is achieved by properly selecting the synaptic weights $J_{ij}$. Adopt a **Hebbian learning rule**, inspired by the principle that co-activated neurons should strengthen their mutual connection. Given a set of $p$ memory patterns $\{ \boldsymbol{\xi}^\mu \}_{\mu=1}^p$, the synaptic weights are defined as:

$$
J_{ij} = \frac{1}{N} \sum_{\mu=1}^p \xi_i^\mu \xi_j^\mu, \quad i \ne j
\tag{1.5}
$$

Each pattern $\xi^\mu \in \{-1, +1\}^N$ represents a full binary configuration of the network and is assumed to be randomly sampled. The couplings $J_{ij}$ encode correlations between neurons across all stored patterns. The diagonal elements $J_{ii}$ are set to zero, reflecting the absence of self-coupling. This formulation ensures that each pattern $\boldsymbol{\xi}^\mu$ becomes a **stable fixed point** of the network dynamics — that is, a local minimum of the Hamiltonian to which the system can converge during recall.

## 1.6 Summary

This is the theoretical and physiological foundation for modeling neural networks as spin-glass systems. Each neuron is represented as a binary spin, and its interaction with other neurons is mediated by synaptic couplings that are symmetric and determined through Hebbian learning. The dynamics of the system are governed by an energy function, which ensures convergence toward stable configurations that represent memorized patterns. In this way, the network performs associative memory by evolving toward attractors in a rugged energy landscape — a concept borrowed directly from disordered systems in statistical mechanics.


# 2. The Generalized Hopfield Model and the Little Model

This section deepens the analysis of neural dynamics by focusing on two specific models: the Hopfield model and the Little model. Both models aim to capture the collective behavior of networks composed of binary-state neurons and are analyzed using tools from the statistical mechanics of disordered systems. These models are explored in the thermodynamic limit, where the number of neurons $N \to \infty$, but the number of stored memory patterns $p$ remains finite.

The key insight presented is that both the Hopfield and Little models are governed by the thermodynamics of infinite-range Ising spin-glass Hamiltonians. Despite differences in update mechanisms, these models share identical equilibrium behavior below a critical temperature $T_c$. Specifically, when the temperature drops below this transition point, both models exhibit a structured set of stable and metastable states. The most prominent among these are the $2p$ **Mattis ground states**, each fully aligned (or anti-aligned) with one of the stored patterns.

Moreover, below approximately $0.46 T_c$, additional metastable states emerge. These correspond to mixtures of the stored memory patterns and possess dynamic stability. These mixed states reflect the complex geometry of the energy landscape introduced by the competition between multiple embedded patterns. The section also generalizes the analysis to cases where the stored patterns are drawn from more complex or non-random distributions, highlighting how the stability structure depends on memory statistics.

## 2.1 Stochastic Dynamics and the Gibbs Distribution

At zero temperature ($T = 0$), the Hopfield model operates using a purely deterministic update rule: each spin that is misaligned with its molecular field flips its state, and the system monotonically descends in energy. This dynamic leads to convergence toward local minima of the Hamiltonian $H$, as defined earlier. To account for thermal noise and stochasticity in biological systems, the model is extended to finite temperature $T = 1/\beta$, resulting in a **generalized Hopfield model** governed by **Glauber dynamics**.

In this finite-temperature setting, the probability of a network being in configuration $\{S\}$ is given by the **Gibbs distribution**:

$$
P\{S\} \propto \exp\left( -\beta H\{S\} \right)
\tag{2.1}
$$

This expression implies that states with lower energy are exponentially more probable, but all configurations can, in principle, occur due to thermal fluctuations. The introduction of temperature allows the model to explore a broader portion of the configuration space and escape shallow local minima. However, it's important to note that **stability under single-spin flips** is no longer sufficient to guarantee dynamic stability in this regime.

## 2.2 Probabilistic Updates in the Little Model

Unlike the Hopfield model, which updates one spin at a time (asynchronously), the **Little model** uses **synchronous updates**, where all spins are updated simultaneously at each discrete time step. The probability that the $i$-th spin takes a new value $S_i'$ at time $t + \delta t$, given the current state of the network $\{\overline{S}_l\}$ at time $t$, is defined by:

$$
P(S_i') = \frac{ \exp(-\beta S_i' h_i) }{ \exp(-\beta S_i' h_i) + \exp(+\beta S_i' h_i) }
\tag{2.2}
$$

Here, $h_i = V_i - U_i = \sum_j J_{ij} S_j$ is the molecular field acting on neuron $i$, as defined previously. This equation arises naturally from Glauber-type dynamics and describes the likelihood of a neuron adopting a particular state based on its input field and the system temperature.

## 2.3 Transition Matrix and System-Wide Dynamics

The **transition probability matrix** $W$ defines the likelihood of transitioning from one global network state $|\alpha, t\rangle$ to another $|\beta, t + \delta t \rangle$. In the Little model, since all spins are updated simultaneously and independently, the total transition probability is given by the product of individual spin update probabilities:

$$
\langle \beta | W | \alpha \rangle = \prod_{l=1}^{N} \left[ \frac{1}{2} e^{-\beta h_l^\alpha S_l^\beta} \, \text{sech}(\beta h_l^\alpha S_l^\beta) \right]
\tag{2.3}
$$

This formulation captures the fact that the entire network evaluates and potentially updates its configuration in parallel at each time step. As a result, multiple spin flips may occur simultaneously, enabling rapid transitions across the energy landscape.

## 2.4 Effective Hamiltonian and Stationary Behavior

A crucial result shown by Peretto is that when the couplings $J_{ij}$ are symmetric, the stochastic dynamics described by the Little model satisfy the condition of **detailed balance**. This ensures that the system's long-term behavior can be described by a stationary distribution that resembles a Boltzmann distribution but with an **effective Hamiltonian** $\overline{H}$, given by:

$$
\overline{H} = -\frac{1}{\beta} \sum_i \ln \left[ 2 \cosh\left( \beta \sum_j J_{ij} S_j \right) \right]
\tag{2.4}
$$

This form of the energy function encapsulates the probabilistic nature of the synchronous update rule. It differs from the original Hamiltonian in that it smooths over rapid fluctuations in the energy landscape, making it suitable for describing equilibrium properties under collective spin dynamics.

Although the synchronous dynamics of the Little model may seem fundamentally different from the asynchronous dynamics of the Hopfield model, both converge to similar thermodynamic behavior under symmetric couplings and in the absence of external fields. Moreover, real biological networks are likely to operate somewhere between these two extremes, incorporating features of both update mechanisms.

Importantly, for networks with Hebbian couplings as defined in Equation (1.5), the Little model does not become trapped in indefinite oscillations or limit cycles. Instead, it reliably converges to aligned, stable states, enabling successful memory retrieval even under synchronous update schemes.

## 2.5 Summary

This provides a rigorous examination of neural network dynamics from both deterministic and stochastic perspectives. By extending the Hopfield model to finite temperature and comparing it with the synchronously-updated Little model, both systems exhibit rich and similar energy landscapes characterized by stable memory attractors. Below a critical temperature $T_c$, the networks stabilize into Mattis ground states aligned with stored memories. At lower temperatures, mixed-state attractors appear, reflecting the subtle interplay between disorder and stability.

Through this analysis, the paper illustrates how **spin-glass theory** provides a powerful and unifying language for understanding the collective behavior of large neural networks — regardless of the specific microscopic dynamics used to update them.


# 3. Relationship to Models of Random Magnets

This section explores the deep and illuminating connection between neural network models—particularly those introduced by Hopfield and Little—and the broader field of disordered magnetic systems in statistical physics. In particular, they focus on **infinite-range spin-glass models**, a class of systems in which each spin interacts with every other spin via couplings that incorporate randomness and frustration.

The Hamiltonian introduced earlier in Equations (1.4) and (1.5), which governs the energy landscape of the Hopfield network, is formally identified as a special instance of an **infinite-range spin-glass model**. In this formulation, every pair of spins $(i, j)$ interacts through a **quenched random exchange** $J_{ij}$, meaning the couplings are fixed (quenched) and chosen from a distribution reflecting the stored memory patterns. This structure captures the essence of long-range disorder and memory retrieval.

Here we contrast with the well-known **Sherrington-Kirkpatrick (SK) model**, which serves as a canonical example of an infinite-range spin glass. In the SK model, each $J_{ij}$ is drawn independently from a Gaussian distribution centered at zero. The randomness in the SK model is uncorrelated and isotropic, leading to a complex energy landscape characterized by a large number of **degenerate ground states**. In the thermodynamic limit $N \to \infty$, the number of metastable configurations grows exponentially, and the system exhibits highly nontrivial **static and dynamic behavior** unlike traditional ferromagnets.

At the opposite end of the spectrum lies a much simpler case: the **Mattis model**, which emerges when only a single pattern is stored (i.e., when $p = 1$ in Equation (1.5)). In this limit, the synaptic couplings become highly structured and can be **"gauged away"**, removing the disorder from the Hamiltonian. Thermodynamically, this transforms the system into a trivial ferromagnet, yielding only two ground states:

$$
\{ S_i \} = \pm \{ \xi_i \}
$$

In these states, every bond $S_i S_j J_{ij}$ is positive, indicating an absence of **frustration**. That is, all interactions can be simultaneously satisfied. The energy landscape is smooth, and retrieval dynamics are straightforward.

The Hopfield model with $p > 1$, however, falls into an **intermediate regime** between these two extremes. In this case, while **a finite fraction of bonds are necessarily frustrated**, the structure of the memory patterns $\{ \xi_i^\mu \}$ introduces correlations among the $J_{ij}$ that give rise to a **less degenerate and more structured broken-symmetry phase** than is seen in the SK model. This suggests that even though the system exhibits frustration, the **correlated disorder** helps stabilize distinct attractors (memories) and prevents complete thermodynamic chaos.

There are closely related models in the literature. One notable example is the **Van Hemmen model** for $p = 2$, which was solved exactly. This model was later generalized by **Provost and Vallee** to handle arbitrary $p$. These mean-field models share structural similarities with the Hopfield model but differ in their assumptions about the distribution and structure of the stored patterns. At the time of this paper's publication, many questions regarding the **detailed organization of metastable states**, as well as the nature of the **mean-field solutions** for arbitrary $p$, remained open. The present paper positions itself as a major contribution toward addressing these questions.

Finally, the section highlights an elegant analogy with the **mean-field theory of random-axis ferromagnets**. In such systems, each spin has a strong **local anisotropy**, defined by a unit vector $\hat{n}_i$, and the coupling between spins is given by the scalar product $J_{ij} \propto \hat{n}_i \cdot \hat{n}_j$. This leads to an effective Ising-like spin glass, where the $J_{ij}$ resemble those in Equation (1.5). However, instead of the binary patterns $\xi_i^\mu \in \{ \pm 1 \}$, the components are drawn from continuous unit vectors. In this framework, the spin-glass-like behavior depends not only on the strength of the couplings but also on the **geometry and statistical distribution** of the orientations $\{ \hat{n}_i \}$. This analogy underscores the broader relevance of the Hopfield-type models, and emphasizes that the **choice of pattern distribution**—whether binary, Gaussian, or vectorial—has profound implications for the system's phase structure and retrieval capabilities.

## 3.1 Summary

This situates the Hopfield and Little models within the rich landscape of spin-glass theory. These neural networks can be understood as specialized instances of infinite-range disordered systems, interpolating between the highly ordered Mattis model and the highly chaotic SK model. The nature of frustration, the structure of quenched disorder, and the symmetries encoded in the memory patterns all influence the complexity of the energy landscape and the stability of stored memories. This synthesis provides a bridge between neuroscience-inspired models and foundational results in statistical physics, revealing deep parallels between learning in neural networks and phase transitions in disordered magnets.


$$
$$